In [ ]:
from google.cloud import bigquery, pubsub_v1
from google.oauth2 import service_account
import requests, json, time, os

project_id = 'persuasive-pipe-384604'
topic_name = 'stock_streaming'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\karth\Documents\key\gcpkey.json"

# Create a client object
client = secretmanager.SecretManagerServiceClient()
secret_id = 'apikey'
version_id = 'latest'  # Use the latest version of the secret
name = f"projects/211234264418/secrets/{secret_id}/versions/{version_id}"
response = client.access_secret_version(request={"name": name})
# Get the secret value
passkey = response.payload.data.decode('UTF-8')


#Create publisher client
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_name)

# Load JSON data

stock_sym = ['AAPL', 'MSFT', 'GOOGL', 'NVDA' ,'META']
while True :
    for symbol in stock_sym:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={passkey}"
        r = requests.get(url)
        stock_data = r.json()
        for timestamp, values in stock_data["Time Series (5min)"].items() :
            row = {
        "symbol" : symbol,
        "timestamp": timestamp,
        "open": float(values["1. open"]),
        "high": float(values["2. high"]),
        "low": float(values["3. low"]),
        "close": float(values["4. close"]),
        "volume": int(values["5. volume"]),
       }
#Publish to topic
            json_string = json.dumps(row)
            print(json_string)
            future = publisher.publish(topic_path, data=json_string.encode('utf-8'))
            print(f"Published message for {symbol} with ID: {future.result()}")
    time.sleep(45)